# Решение дифференциальных уравнений в sympy

Рассмотрим решение обыкновенных дифференциальных уравнений (ОДУ) в sympy. Это дифференциальное уравнение для функции от одной переменной.

По-английски это "ordinary differential equations (ODE)"

Раньше основой работы в sympy были символы в смысле "неизвестное значение". В решении ODE нужно ввести новое понятие "неизвестная функция".

Начнем с import

In [1]:
from sympy import *

и определения переменных, которые могут пригодиться:

In [2]:
x, y, z = symbols('x y z')

## Определим неизвестную функцию

Введем понятие "функция". Можно ввести фукнцию неизвестно от какого аргумента:

In [5]:
f1 = Function('f')
f1

f

Функция недоопределена. Неизвестно, от какого она аргумента.

In [6]:
type(f1)

sympy.core.function.UndefinedFunction

In [8]:
f1(x)   # так нужно потом указывать, что аргумент х

f(x)

Удобнее сразу определять от каких аргументов будет функция. Например, $g(x, y, z)$

In [9]:
g = Function('g')(x, y, z)
g

g(x, y, z)

In [12]:
g.free_symbols   # узнаем, какие у функции аргументы

{x, y, z}

## Определим производные неизвестной функции

Вспомним, как мы находили производные выражений. Нужна функция **diff**

In [14]:
ex = x**3
diff(ex)

3*x**2

Вторая производная и производная порядка n:

**diff(функция, переменная, порядок)** - производная от функции по переменной указанного порядка 

In [15]:
diff(x**3, x, 2)   # вторая производная

6*x

In [22]:
diff(g,x)

Derivative(g(x, y, z), x)

или пишем как для черепахи: `объект.метод(аргументы)`

In [23]:
g.diff(x)

Derivative(g(x, y, z), x)

Вторая и выше производная задается еще одним аргументом одним из двух способов:

In [24]:
diff(g, x, 2)    # функция

Derivative(g(x, y, z), (x, 2))

In [25]:
g.diff(x, 2)     # метод

Derivative(g(x, y, z), (x, 2))

## Запишем дифференциальное уравнение и решим его

Задача: Решите уравнение $y''+y' -6y= 0$

Решение: сначала нужно объявить переменную и функцию от этой переменной.

In [26]:
x = symbols('x')         # переменная х
y = Function('y')(x)     # функция y(x)
y                        # проверяем, что определили правильно

y(x)

Уравнение пишем как раньше, используя **Eq** (как раньше), производные пишем через **diff**:

In [27]:
q = Eq(diff(y, x, 2) + diff(y, x) -6*y, 0)   # пишем левую и правую части уравнения
q                                            # обязательно проверяем, что написали правильно

Eq(-6*y(x) + Derivative(y(x), x) + Derivative(y(x), (x, 2)), 0)

Раньше для решения уравнений использовали **solve**. Теперь решаем дифференциальные уравнения. Для решения используем **dsolve**.

In [29]:
sol = dsolve(q)
sol

Eq(y(x), C1*exp(-3*x) + C2*exp(2*x))

$C_1$ и $C_2$ - это символы. Они уже созданы. Мы их не можем найти, пока не заданы *граничные условия*

## Решение уравнения с граничными условиями

Пусть в предыдущем уравнении заданы граничные условия и полностью задача такая:

Решите уравнение $y''+y' -6y= 0$ при $y(0) = 1$ и $y'(0) = 0$

Тогда мы можем найти значения $C_1$ и $C_2$ в решении $\displaystyle y{\left(x \right)} = C_{1} e^{- 3 x} + C_{2} e^{2 x}$

В выражении `sol` есть левая часть уравнения $y(x)$ и правая часть $C_{1} e^{- 3 x} + C_{2} e^{2 x}$, которые можно получить из `sol` как `sol.`**lhs** и `sol.`**rhs** (left hands и right hands). Нам нужна правая часть:

In [33]:
sol.rhs

C1*exp(-3*x) + C2*exp(2*x)

Запишем эту правую часть в выражение fc

In [38]:
fc = sol.rhs
fc

C1*exp(-3*x) + C2*exp(2*x)

$y(0) = 1$ это уравнение. Найдем сначала $y(0)$, подставим $x=0$ в $C_{1} e^{- 3 x} + C_{2} e^{2 x}$

In [39]:
fc.subs({x:0})

C1 + C2

то есть $y(0) = 1$ это

In [40]:
q1 = Eq(fc.subs({x:0}), 1)
q1

Eq(C1 + C2, 1)

Запишем второе граничное условие $y'(0) = 0$, возьмем производную от выражения `fc`. Запишем результат в перменную `dfc`

In [41]:
dfc = diff(fc, x)
dfc

-3*C1*exp(-3*x) + 2*C2*exp(2*x)

Запишем второе граничное условие как уравнение

In [42]:
q2 = Eq(dfc.subs({x:0}), 0)
q2

Eq(-3*C1 + 2*C2, 0)

У нас получилась система обычных (НЕ диференциальных) уравнений `q1` и `q2`:

$$\begin{cases}
C_{1} + C_{2} = 1\\
-3 C_{1} + 2 C_{2} = 1
\end{cases}$$

Их нужно решить **относительно переменных $C_1$ и $C_2$**, а не относительно х. Тут нигде нет переменной х.

In [44]:
csol = solve([q1, q2], dict=True)
csol

[{C1: 2/5, C2: 3/5}]

* Это система обычных уравнений, а не дифференциальных, поэтому **solve**, а не dsolve.
* sympy догадывается, что если в уравнениях есть переменные, то решать нужно относительно них
* нужно будет подставлять значения C1 и C2, поэтому решение получаем в виде подстановки `dict=True`

Подставим найденные значения констант в решение дифференциального уравнения и получим ответ:

In [46]:
fres = fc.subs(csol[0])
fres

3*exp(2*x)/5 + 2*exp(-3*x)/5

## Проверим решение ОДУ

Вычислим значение выражения $y'' + y' - 6 y$ для $$y(x) = \frac{3 e^{2 x}}{5} + \frac{2 e^{- 3 x}}{5}$$

In [47]:
ex = diff(fres, x, 2) + diff(fres, x) - 6*fres
ex

6*(2*exp(2*x) + 3*exp(-3*x))/5 - 12*exp(2*x)/5 - 18*exp(-3*x)/5

In [50]:
simplify(ex)   # упростим выражение, ожидаем получить 0

0

Проверим граничные условия

In [51]:
fres.subs({x:0})          # y(0) = 1

1

In [52]:
diff(fres).subs({x:0})    # y'(0) = 0

0